In [49]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, auc, roc_curve, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer, KBinsDiscretizer, LabelEncoder, MinMaxScaler, PowerTransformer

import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use("ggplot")      
import seaborn as sns
# import missingno as msno
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


# from pandas_profiling import ProfileReport
# ProfileReport(train).to_notebook_iframe()

In [50]:
# 数据读取，删除无效列

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test1.csv')

train.drop(['Unnamed: 0'],axis=1,inplace=True)
test.drop(['Unnamed: 0'],axis=1,inplace=True)
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,1,NaN,104,6.0,android,9,18,1438873,1.559893e+12,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,1,NaN,19,6.0,android,8.1,0,1185582,1.559994e+12,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,1,NaN,559,0.0,android,8.1.0,0,1555716,1.559837e+12,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,0,NaN,129,2.0,android,8.1.0,0,1093419,1.560042e+12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,2.0,android,8.0.0,0,1400089,1.559867e+12,5,2364522023,4,1510695983,582


In [51]:
train.describe()
train.columns,len(train.columns)-1

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,media_id,ntt,package,sid,timestamp,location,fea1_hash,cus_type
count,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.00000,500000.00000,500000.000000,500000.000000,5.000000e+05,5.000000e+05,500000.000000,5.000000e+05,500000.000000
mean,261359.275126,986.640110,40028.788034,1264.986626,72.027966,703.486166,0.48448,124.08762,3.089808,38.465876,1.500335e+06,1.559814e+12,96.040504,2.300866e+09,730.824682
std,233616.172774,128.956348,15460.788899,853.371330,167.664930,505.751343,0.49976,164.25454,1.843088,136.321129,2.884292e+05,1.680735e+08,85.652740,1.236593e+09,331.946854
min,0.000000,95.000000,-1.000000,0.000000,0.000000,0.000000,0.00000,4.00000,0.000000,0.000000,1.000005e+06,1.559491e+12,-1.000000,1.240000e+04,297.000000
25%,0.000000,917.000000,46000.000000,720.000000,0.000000,360.000000,0.00000,29.00000,2.000000,0.000000,1.250850e+06,1.559664e+12,23.000000,1.376752e+09,411.000000
50%,228563.000000,1001.000000,46000.000000,1280.000000,0.000000,720.000000,0.00000,64.00000,2.000000,7.000000,1.500358e+06,1.559816e+12,64.000000,2.490131e+09,658.000000
75%,465701.500000,1076.000000,46000.000000,2040.000000,0.000000,1080.000000,1.00000,139.00000,5.000000,24.000000,1.750028e+06,1.559964e+12,154.000000,3.062465e+09,1019.000000
max,709898.000000,1241.000000,46003.000000,9024.000000,720.000000,8832.000000,1.00000,1544.00000,7.000000,2327.000000,1.999999e+06,1.560096e+12,330.000000,4.291920e+09,1380.000000


(Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
        'dev_width', 'label', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
        'sid', 'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash',
        'cus_type'],
       dtype='object'),
 19)

In [52]:
# 把各个特征的分布都用直方图画一下(除了fea_hash、fea1_hash、label)

# fea_plt = train.columns.tolist()
# fea_plt.remove('label')
# fea_plt.remove('fea_hash')
# fea_plt.remove('fea1_hash')

# for i in range(len(fea_plt)):
#     print('{}'.format(fea_plt[i]))
#     train[fea_plt[i]].hist()
#     plt.show();
# print(i)    



# plt.figure()
# fig, axs = plt.subplots(4, 5, figsize=(18, 6))

# n = 0
# for i in range(4):
#     x=0
#     for j in range(5):   
#         axs[i][j].hist(train[fea_plt[x+n*5]])
#         axs[i][j].set_title('{}'.format(fea_plt[x+n*5]))
#         x+=1
#     n+=1

# fig.tight_layout();


In [53]:
# plt.figure(figsize=(12, 10))
# sns.heatmap(train.corr(),linewidths = 0.05);

# # 少了fea_hash，发现这列是str，并且有些奇怪的值
# print('与label的相关性：')
# train.corr().label.sort_values()

In [54]:
# # 特征取值类别

# for col in train.columns:
#     col,train[col].nunique()

In [55]:
data = pd.concat([train,test])

data.osv.unique()

array(['9', '8.1', '8.1.0', '8.0.0', '5.1', '9.0.0', '7.1.1', '5.1.1',
       '7.8.7', '4.4.4', nan, '6.0', '6.0.1', '3.9.0', 'Android_8.1.0',
       '7.0', '7.9.2', 'Android_9', '7.9.0', '7.1.2', '3.8.6', '5.1.0',
       '9.0', '7.7.7', '6.0.0', '5.0.2', 'Android_6.0.1', '7.0.0',
       '6.0_23', '4.2.2', '7.1', '8.0', '4.3', 'Android_8.0.0', '4.4.2',
       '4.1', 'Android_7.1.1', '5.0', 'Android_5.1.1', '2.3', '5.0.0',
       '7.8.0', '5.0.1', '4.4', '7.8.6', 'Android_5.1', 'Android_7.1.2',
       'Android_4.4.4', '8.0.1', '3.1.1', '7.7.2', '6.1', '3.8.4',
       '7.8.9', '7.7.5', '7.6.8', 'Android_7.0', '7.8.4', '7', '7.7.3',
       '7.1.0', '9.1', '4.1.1', 'Android_4.4.2', '3.8.0', '6.1.2',
       '7.2.1', '7.6.9', '3.7.8', '7.8.2', '8', '7910', '7.6.4', '7.8.5',
       '2.2.3', '5.2', 'Android_6.0', '4.2.1', '10.3.3', '4.0.2', '7.8.8',
       '4.4.3', '21100', '7.7.0', '4.0.3', '4.0.4', '5', '2.3.6', '6.1.0',
       '21000', '4.3.0', '4.1.2', '4.2', '7.6.7', '9.0.5', '4.4.2_19',


## 特征工程

In [56]:
def handle_osv(osv):
    osv = str(osv).replace('.','').replace('W','').replace('Android_','').replace('Android', '').replace('十核20G_HD', '').replace(' ','')
    if osv=='nan' or osv=='GIONEE_YNGA': # 这2个值的前面的取值都为8.1.0
        result = 810
    elif osv=='f073b_changxiang_v01_b1b8_20180915':
        result = 810
    elif osv=='%E6%B1%9F%E7%81%B5OS+50':
        result=500
    elif osv.count('-')>0:
        result = int(osv.split('-')[0])
    else:
        result = int(osv)
    
    if result<10:
        result*=100
    elif result<100:
        result*=10
    
    return int(result)


data['osv'] = data['osv'].apply(handle_osv)     
data['osv'].unique()

array([  900,   810,   800,   510,   711,   511,   787,   444,   600,
         601,   390,   700,   792,   790,   712,   386,   777,   502,
        6023,   422,   710,   430,   442,   410,   500,   230,   780,
         501,   440,   786,   801,   311,   772,   610,   384,   789,
         775,   768,   784,   773,   910,   411,   380,   612,   721,
         769,   378,   782,  7910,   764,   785,   223,   520,   421,
        1033,   402,   788,   443, 21100,   770,   403,   404,   236,
       21000,   412,   420,   767,   905, 44219,   383,   433,   445,
         376,   235,   713,   303,   621,  5122, 60119,  7930,  4232,
         512,   774,   602,   431,   237,   530, 71200,   212,   766,
         234,   320,   110, 51122,   400,   731,   802,  5021,   201,
         429,  7920,   503,   292,   222, 60122,   446, 71300,   120],
      dtype=int64)

In [57]:
def handle_version(version):
    version = str(version).replace(' ','').replace('v','').replace('V','').replace('GA','').replace('P_Final_','').replace('GA','')
    # 发现version=50附近都是5
    if version == '50':
        return int(5)
    return int(version)

data['version'] = data['version'].apply(handle_version)
data['version'].unique()
    

array([ 8,  4,  0,  5,  9,  7,  3,  1,  6, 11,  2, 10, 15, 20],
      dtype=int64)

In [58]:
def foreign_lan(x):
    set1 = {'zh-CN', 'zh', 'cn', 'zh_CN', 'Zh-CN', 'zh-cn', 'ZH', 'CN', 'zh_CN_#Hans'}
    if x in set1:
        return 0
    elif x=='unk':
        return 2
    else:
        return 1
    
data["vpn"] = data["lan"].apply(foreign_lan)
data["vpn"] = data["lan"].apply(foreign_lan)

In [59]:
from sklearn.preprocessing import LabelEncoder

LabelEncoder= LabelEncoder()
data['lan']=LabelEncoder.fit_transform(data['lan'].astype('str'))

data['lan'].unique()

array([14, 17, 16,  4, 22, 23,  3, 15,  2,  5,  0,  6,  1, 24, 18,  7, 20,
       12, 19, 10, 13, 11,  8, 21,  9])

In [60]:
import datetime

data['timestamp'] = data['timestamp'].apply(lambda x : datetime.datetime.fromtimestamp(x/1000))

data['year']= data['timestamp'].dt.year
data['month']= data['timestamp'].dt.month
data['day']= data['timestamp'].dt.day
data['hour']= data['timestamp'].dt.hour
data['minute']= data['timestamp'].dt.minute
data['weekday']= data['timestamp'].dt.weekday

# 划分训练集
train = data[data['label'].notnull()]
test = data[data['label'].isnull()].drop(['label'],axis=1)

# 获取与最小时间的时间差
start_time1 = train['timestamp'].min()
train['timestamp_diff'] = train['timestamp']-start_time1
train['timestamp_diff'] = train['timestamp_diff'].dt.days - train['timestamp_diff'].dt.seconds/3600/24

start_time2 = test['timestamp'].min()
test['timestamp_diff'] = test['timestamp']-start_time2
test['timestamp_diff'] = test['timestamp_diff'].dt.days - test['timestamp_diff'].dt.seconds/3600/24

data = pd.concat([train,test])

In [61]:
# fea_hash 有5w多个不同的取值,大多数就10位，大于10位的就处理成0

train['fea_hash'].apply(len).quantile([.1,.2,.5,.75,.95])
data['fea_hash'] = data['fea_hash'].map(lambda i:0 if len(str(i))>10 else int(i))
data['fea_hash_len'] = data['fea_hash'].map(lambda i:len(str(i)))

0.10     9.0
0.20     9.0
0.50    10.0
0.75    10.0
0.95    10.0
Name: fea_hash, dtype: float64

In [62]:
# fea1_hash 大多数就10位，大于10位的就处理成-1

train['fea1_hash'].astype('str').map(len).quantile([.1,.2,.5,.75,.95])
data['fea1_hash'] = data['fea1_hash'].map(lambda i:0 if len(str(i))>10 else int(i))
data['fea1_hash_len'] = data['fea1_hash'].map(lambda i:len(str(i)))

0.10     9.0
0.20     9.0
0.50    10.0
0.75    10.0
0.95    10.0
Name: fea1_hash, dtype: float64

In [63]:
def divided(x):
    if x % 40 == 0:
        return 2
    elif not x:
        return 1
    else:
        return 0
        
data["160_height"] = data.dev_height.apply(divided)
data["160_width"] = data.dev_width.apply(divided)
data["hw_ratio"] = data.dev_height / data.dev_width
data["hw_matrix"] = data.dev_height * data.dev_width
data['osv_ver'] = data['osv'] - data['version']

In [64]:
data.head()
data.shape
data.columns

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,...,minute,weekday,timestamp_diff,fea_hash_len,fea1_hash_len,160_height,160_width,hw_ratio,hw_matrix,osv_ver
0,316361,1199,46000.0,0.0,0.0,0.0,1.0,14,104,6.0,...,32,4,3.352697,10,10,2,2,NaN,0.0,892
1,135939,893,0.0,0.0,0.0,0.0,1.0,14,19,6.0,...,40,5,4.180336,10,10,2,2,NaN,0.0,806
2,399254,821,0.0,760.0,0.0,360.0,1.0,14,559,0.0,...,58,3,2.001204,10,9,2,2,2.111111,273600.0,810
3,68983,1004,46000.0,2214.0,0.0,1080.0,0.0,14,129,2.0,...,59,6,5.625278,10,10,0,2,2.050000,2391120.0,810
4,288999,1076,46000.0,2280.0,0.0,1080.0,1.0,17,64,2.0,...,28,4,3.646991,10,10,2,2,2.111111,2462400.0,795


(650000, 35)

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'label', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
       'sid', 'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash',
       'cus_type', 'vpn', 'year', 'month', 'day', 'hour', 'minute', 'weekday',
       'timestamp_diff', 'fea_hash_len', 'fea1_hash_len', '160_height',
       '160_width', 'hw_ratio', 'hw_matrix', 'osv_ver'],
      dtype='object')

## 模型预测

In [65]:
feat_cols = ['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
             'dev_width', 'lan', 'media_id', 'ntt', 'osv', 'package',
             'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
             'fea_hash_len', 'fea1_hash_len', 'vpn', 'year', 'month', 'day', 'weekday',
             'hour', 'minute', 'timestamp_diff', 'osv_ver', '160_height',
             '160_width','hw_matrix','hw_ratio']

print('参与训练的特征个数：{}'.format(len(feat_cols)))

train = data[data['label'].notnull()]
test = data[data['label'].isnull()].drop(['label'],axis=1)
print('训练集、测试集已准备~')

参与训练的特征个数：31
训练集、测试集已准备~


In [66]:
lgb_model = lgb.LGBMClassifier(objective= 'binary',
                               metric= 'auc',
                               num_leaves= 512,  # num_leaves < 2^max_depth
                               boosting_type= 'gbdt',
                               bagging_freq= 1,
                               lambda_l1= 0.5,
                               lambda_l2= 0.5,
                               n_estimators= 5000,
                               learning_rate= 0.005,
                               feature_fraction= 0.8,
                               bagging_fraction= 0.8,
                               max_depth= 12,
                               n_jobs= -1,
                               random_state= 2021
                               )

# 划分训练集
X = train[feat_cols]
y = train['label']

# 5折-交叉验证
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2022)
pred = []
for folder,(train_index, test_index) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    lgb_model = lgb_model.fit(X_train,
                              y_train,
                              eval_set=[(X_val, y_val)],
                              eval_metric='auc',
                              verbose=True)
    # 预测
    test_pre = lgb_model.predict_proba(test[feat_cols])
    print('第{}轮训练结束，正在保存预测数据-------------------------'.format(folder+1))
    pred.append(test_pre)
    


[1]	valid_0's auc: 0.93295
[2]	valid_0's auc: 0.937975
[3]	valid_0's auc: 0.939826
[4]	valid_0's auc: 0.94041
[5]	valid_0's auc: 0.940717
[6]	valid_0's auc: 0.941027
[7]	valid_0's auc: 0.941082
[8]	valid_0's auc: 0.941201
[9]	valid_0's auc: 0.941168
[10]	valid_0's auc: 0.941152
[11]	valid_0's auc: 0.941142
[12]	valid_0's auc: 0.941201
[13]	valid_0's auc: 0.941192
[14]	valid_0's auc: 0.941289
[15]	valid_0's auc: 0.941247
[16]	valid_0's auc: 0.941228
[17]	valid_0's auc: 0.941269
[18]	valid_0's auc: 0.941273
[19]	valid_0's auc: 0.941275
[20]	valid_0's auc: 0.941334
[21]	valid_0's auc: 0.941355
[22]	valid_0's auc: 0.941352
[23]	valid_0's auc: 0.941361
[24]	valid_0's auc: 0.941346
[25]	valid_0's auc: 0.941376
[26]	valid_0's auc: 0.941302
[27]	valid_0's auc: 0.941262
[28]	valid_0's auc: 0.941279
[29]	valid_0's auc: 0.941309
[30]	valid_0's auc: 0.941357
[31]	valid_0's auc: 0.941345
[32]	valid_0's auc: 0.941365
[33]	valid_0's auc: 0.941427
[34]	valid_0's auc: 0.941464
[35]	valid_0's auc: 0.941

In [69]:
xgb_model = xgb.XGBClassifier(max_depth=15,
                              learning_rate=0.005,
                              n_estimators=1000,
                              objective='binary:logistic',
                              subsample=0.8, # Subsample ratio of the training instance.
                              colsample_bytree=0.8, # Subsample ratio of columns when constructing each tree.
                              min_child_samples=3, 
                              eval_metric='auc',
                              random_state=2022,
                              reg_alpha =0.5, # L1 regularization
                              reg_lambda  = 0.5 # L2
                              )

# 5折-交叉验证
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2022)
pre2 = []
for kfolder,(train_index, test_index) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    xgb_model = xgb_model.fit(X_train,
                              y_train,
                              eval_set=[(X_val, y_val)],
                              eval_metric='auc',
                              verbose=True)
    
    acc =accuracy_score(y_val,xgb_model.predict(X_val))
    print('第{}轮，验证集accuracy={}'.format(kfolder+1,acc))
    # 预测
    test_pre2=xgb_model.predict_proba(test[feat_cols])
    print('第{}轮训练结束，正在保存预测数据-------------------------'.format(folder+1))
    pre2.append(test_pre2)

[15:44:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "min_child_samples" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.93912
[1]	validation_0-auc:0.94158
[2]	validation_0-auc:0.94254
[3]	validation_0-auc:0.94310
[4]	validation_0-auc:0.94321
[5]	validation_0-auc:0.94338
[6]	validation_0-auc:0.94348
[7]	validation_0-auc:0.94358
[8]	validation_0-auc:0.94342
[9]	validation_0-auc:0.94351
[10]	validation_0-auc:0.94363
[11]	validation_0-auc:0.94380
[12]	validation_0-auc:0.94386
[13]	validation_0-auc:0.94384
[14]	validation_0-auc:0.94387
[15]	validation_0-auc:0.94385
[16]	validation_0-auc:0.94386
[17]	validation_0-auc:0.94389
[18]	validation_0-auc:0.94391
[19]	validatio

In [125]:
def result_mean(pred,nfolder=5,class_num=2):
    result_prob = np.zeros((test.shape[0],class_num))
    for folder in range(0,nfolder):
        result_prob+=pred[folder]
    result_prob = result_prob/nfolder
    return result_prob

# 结果 
lgb_result = result_mean(pred)
xgb_result = result_mean(pre2)

# 模型融合
result_submit = test[['sid']]
out_put=[]
for i in range(test.shape[0]):
    merge_result = (lgb_result[i][0]+xgb_result[i][0])/2
    out_put.append(np.where(merge_result>0.5,0,1))
result_submit['label'] = out_put

#输出预测结果
result_submit.to_csv('./result.csv', index=False)
print('预测结果已输出~')

预测结果已输出~


In [67]:
# def gen_thres_new(df_train, oof_preds):
#     df_train['oof_preds'] = oof_preds
#     thres = df_train['label'].mean() 
#     # thres = df_train['oof_preds'].quantile(1 - quantile_point) # 比如 0,1,1,1 mean=0.75 1-mean=0.25,也就是25%分位数取值为0

#     _thresh = []
#     for thres_item in np.arange(thres - 0.2, thres + 0.2, 0.01): #  按照理论阈值的上下0.2范围，0.01步长，找到最佳阈值，f1分数最高对应的阈值即为最佳阈值
#         _thresh.append(
#             [thres_item, f1_score(df_train['label'], np.where(oof_preds > thres_item, 1, 0), average='macro')])

#     _thresh = np.array(_thresh)
#     best_id = _thresh[:, 1].argmax() # 找到f1最高对应的行
#     best_thresh = _thresh[best_id][0] # 取出最佳阈值

#     print("阈值: {}\n训练集的f1: {}".format(best_thresh, _thresh[best_id][1]))
#     return best_thresh

# lgb_thres =  gen_thres_new(train, oof_preds_lgb)
# print(lgb_thres)



# # 测试集预测结果
# df_test_res = pd.DataFrame({'sid': test['sid'],
#                             'test_preds_lgb': test_preds_lgb})

# ## 直接按 0.5 划分
# # df_test_res['preds'] =  np.where(df_test_res['test_preds_lgb']>0.5,1,0)


# ## 按最佳阈值划分
# df_test_res['preds'] =  np.where(df_test_res['test_preds_lgb']>lgb_thres,1,0)


# df_test_submit = df_test_res.iloc[:,[0,2]].rename(columns={'preds':'label'})
# df_test_submit.to_csv('./result.csv', index=False)
# print('预测结果，输出成功~')

